In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("muhammadqasimshabbir/amini-soil-prediction-challenge-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/amini-soil-prediction-challenge-dataset


In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import TweedieRegressor

import lightgbm as lgb
import gc
import warnings
from datetime import timedelta, datetime
import itertools

warnings.filterwarnings('ignore')

# --- Configuration ---
BASE_PATH = '/kaggle/input/amini-soil-prediction-challenge-dataset/'
TARGET_NUTRIENTS = ['N', 'P', 'K', 'Ca', 'Mg', 'S', 'Zn', 'Mn', 'Fe', 'Cu', 'B']
SOIL_DEPTH_CM = 20
SEED = 42
K_FOR_KNN_FEATURES = 3

REFERENCE_YEAR = 2019
REFERENCE_END_DATE_OF_MAIN_WINDOW_STR = f"{REFERENCE_YEAR}-08-01"
SATELLITE_TIME_WINDOWS = [
    ("win30d_recent", 30, 0), ("win90d_mid", 90, 30),
    ("win180d_early", 180, 90), ("win_full_year", 365, 0)
]

# Outlier Capping Config
FEATURES_FOR_OUTLIER_CAPPING = [
    'ls', 'soc20', 'slope', 'mb1', 'mb2', 'mb3', 'mb7', 'parv', 'hp20',
]
LOWER_PERCENTILE_CAP = 0.01
UPPER_PERCENTILE_CAP = 0.99

# Ensemble Weights
WEIGHT_LGBM = 0.7
WEIGHT_GLM = 0.3

# --- Pseudo-Labeling Configuration (Using Simpler Version Parameters) ---
ENABLE_PSEUDO_LABELING_LGBM = True
PSEUDO_LABELING_ITERATIONS = 3
PSEUDO_LABELING_CONFIDENCE_PERCENTILE = 80 # As in your simpler working example
PSEUDO_LABELING_N_BOOTSTRAP_CONF = 3 # As in your simpler working example
PSEUDO_LABELING_MIN_SAMPLES = 10 # As in your simpler working example

# --- Pseudo-Labeling Helper Functions (Simpler Version from your "Working Code") ---
def calculate_confidence_lgbm_bootstrap(base_model_params,
                                        X_train_df_conf, y_train_series_conf,
                                        X_test_df_conf,
                                        selected_features,
                                        n_bootstrap_models=3, seed_offset=100, base_seed=42):
    predictions_bootstrap = []
    
    X_train_subset_np = X_train_df_conf[selected_features].values
    y_train_subset_np = y_train_series_conf.values
    X_test_subset_np = X_test_df_conf[selected_features].values

    if X_train_subset_np.shape[0] == 0:
        print("      Warning: Empty training data for bootstrap. Returning zero confidence.")
        return np.zeros(X_test_subset_np.shape[0]) # Return only confidence for this simpler PL

    for i in range(n_bootstrap_models):
        n_samples = X_train_subset_np.shape[0]
        # Standard unweighted bootstrap
        bootstrap_indices = np.random.choice(np.arange(n_samples), size=n_samples, replace=True)
        
        X_boot = X_train_subset_np[bootstrap_indices]
        y_boot = y_train_subset_np[bootstrap_indices]
        
        model_boot_params = base_model_params.copy()
        model_boot_params['seed'] = base_seed + seed_offset + i
        model_boot_params['verbose'] = -1
        
        model_boot = lgb.LGBMRegressor(**model_boot_params)
        try:
            model_boot.fit(X_boot, y_boot) # No sample_weight here for simpler PL
            preds_boot = model_boot.predict(X_test_subset_np)
        except Exception as e_boot:
            print(f"      Warning: Bootstrap model {i+1} fit/predict failed: {e_boot}. Using zeros for this bootstrap.")
            preds_boot = np.zeros(X_test_subset_np.shape[0])
        predictions_bootstrap.append(preds_boot)

    pred_array = np.array(predictions_bootstrap)
    
    if pred_array.shape[0] < 2 and n_bootstrap_models > 0:
        print("      Warning: Not enough successful bootstrap models for std_dev. Returning low confidence.")
        return np.full(X_test_subset_np.shape[0], 1e-6) # Low confidence
    elif n_bootstrap_models == 0:
         return np.ones(X_test_subset_np.shape[0]) # Neutral confidence

    std_dev = np.std(pred_array, axis=0)
    confidence = 1.0 / (std_dev + 1e-6)
    return confidence # Return only confidence

def confident_pseudo_labeling_lgbm( # This is the simpler version
    initial_model_params,
    X_train_orig_df, y_train_orig_series,
    X_test_orig_df, # This is the features of the test set where we want to generate pseudo-labels
    selected_features_list,
    iterations=5,
    confidence_percentile=80,
    n_bootstrap_for_confidence=3,
    base_seed=42,
    min_confident_samples_to_add=1
):
    current_X_train_df = X_train_orig_df.copy()
    current_y_train_series = y_train_orig_series.copy()
    # No sample weights in this simpler version

    print(f"    Starting pseudo-labeling. Initial train size: {len(current_X_train_df)}")

    for i in range(iterations):
        print(f"      Pseudo-labeling iteration {i+1}/{iterations}")
        
        iter_model_params = initial_model_params.copy()
        current_model_for_preds = lgb.LGBMRegressor(**iter_model_params)
        # Fit on current training data (no weights)
        current_model_for_preds.fit(
            current_X_train_df[selected_features_list], 
            current_y_train_series
        )
        
        # Predict on original test data (X_test_orig_df) to get potential pseudo-labels
        test_preds = current_model_for_preds.predict(X_test_orig_df[selected_features_list])
        
        # Calculate prediction confidence using bootstrap models trained on current_X_train_df
        confidence = calculate_confidence_lgbm_bootstrap(
            initial_model_params,
            current_X_train_df, # Current augmented training data
            current_y_train_series, # Current augmented training labels
            X_test_orig_df,       # Original test data for confidence estimation
            selected_features_list,
            n_bootstrap_models=n_bootstrap_for_confidence,
            seed_offset=1000 * (i + 1),
            base_seed=base_seed
        )
        
        finite_confidence = confidence[np.isfinite(confidence)]
        if len(finite_confidence) == 0:
            print("        All confidence values are non-finite. Stopping pseudo-labeling.")
            break
        
        confidence_threshold = np.percentile(finite_confidence, confidence_percentile)
        # Create mask based on X_test_orig_df's length
        confident_mask = (confidence >= confidence_threshold) & np.isfinite(confidence)
        
        num_confident_samples = np.sum(confident_mask)

        if num_confident_samples < min_confident_samples_to_add:
            print(f"        Found only {num_confident_samples} confident pseudo-labels (min required: {min_confident_samples_to_add}). Stopping.")
            break
        
        print(f"        Found {num_confident_samples} confident pseudo-labels (percentile: {confidence_percentile}th).")

        # Get pseudo-labeled data (features from X_test_orig_df, labels are test_preds)
        X_pseudo_df = X_test_orig_df[selected_features_list][confident_mask].copy()
        # Create a Series for pseudo labels, ensuring correct indexing from X_pseudo_df
        y_pseudo_series = pd.Series(test_preds[confident_mask], index=X_pseudo_df.index, name=current_y_train_series.name)
        
        # Add confident pseudo-labels to the training set
        # ignore_index=True is critical for this simpler version as it avoids index conflicts
        current_X_train_df = pd.concat([current_X_train_df, X_pseudo_df], ignore_index=True)
        current_y_train_series = pd.concat([current_y_train_series, y_pseudo_series], ignore_index=True)
        
        print(f"        New train size: {len(current_X_train_df)}")
        gc.collect()

    final_model_params = initial_model_params.copy()
    final_model = lgb.LGBMRegressor(**final_model_params)
    print(f"    Fitting final model on augmented data of size {len(current_X_train_df)}")
    final_model.fit(
        current_X_train_df[selected_features_list], 
        current_y_train_series # No sample_weight
    )
    
    return final_model


# --- Satellite Data Processing Function (QA Masking Implemented) ---
def load_and_preprocess_satellite_data_multi_window_knn_features(
    filepath, all_pids_from_base, reference_end_date_main_str,
    time_windows_config, k_neighbors
):
    # print(f"Processing satellite data (Temporal Aggregates, k={k_neighbors}): {filepath.split('/')[-1]}") # Less verbose
    try:
        sat_df_full = pd.read_csv(filepath)
    except FileNotFoundError:
        # print(f"File not found: {filepath}. Returning empty placeholder.")
        return pd.DataFrame(index=pd.Index(all_pids_from_base, name='PID'))

    filename_prefix = filepath.split('/')[-1].split('.')[0].replace('_data_updated','').replace('_data','')
    
    date_col_sat = None
    possible_date_cols = ['Date', 'system:time_start', 'DATE_ACQUIRED', 'sensing_time', 'date']
    for col in possible_date_cols:
        if col in sat_df_full.columns:
            date_col_sat = col
            break
    if date_col_sat is None:
        # print(f"No date column found in {filepath}. Returning empty placeholder.")
        return pd.DataFrame(index=pd.Index(all_pids_from_base, name='PID'))

    if sat_df_full[date_col_sat].dtype == 'object' or pd.api.types.is_string_dtype(sat_df_full[date_col_sat]):
        try: sat_df_full[date_col_sat] = pd.to_datetime(sat_df_full[date_col_sat], errors='raise')
        except: sat_df_full[date_col_sat] = pd.to_datetime(sat_df_full[date_col_sat].astype(str).str.split('T').str[0], errors='coerce')
    elif pd.api.types.is_numeric_dtype(sat_df_full[date_col_sat]):
        try: sat_df_full[date_col_sat] = pd.to_datetime(sat_df_full[date_col_sat], unit='ms', errors='coerce')
        except: sat_df_full[date_col_sat] = pd.to_datetime(sat_df_full[date_col_sat], errors='coerce')
    
    if sat_df_full[date_col_sat].isnull().all():
        # print(f"All date values are null in {filepath}. Returning empty placeholder.")
        return pd.DataFrame(index=pd.Index(all_pids_from_base, name='PID'))

    # Ensure PID is string for consistent merging from the start
    sat_df_full['PID'] = sat_df_full['PID'].astype(str)
    all_pids_from_base_str = [str(p) for p in all_pids_from_base]


    sat_df_filtered_base = sat_df_full.dropna(subset=[date_col_sat, 'PID'])
    # Filter by PIDs present in the main dataset (all_pids_from_base_str)
    sat_df_filtered_base = sat_df_filtered_base[sat_df_filtered_base['PID'].isin(all_pids_from_base_str)].copy()


    if sat_df_filtered_base.empty:
        # print(f"No data for relevant PIDs in {filepath}. Returning empty placeholder.")
        return pd.DataFrame(index=pd.Index(all_pids_from_base_str, name='PID')) # Use string PIDs for index

    ndvi_base_name = "NDVI_calc"
    # Define potential_base_feature_names before QA masking, based on original columns
    potential_base_feature_names = [
        c for c in sat_df_full.columns 
        if c not in ['PID', date_col_sat, 'QA_PIXEL', 'SCL'] # Exclude QA columns too
        and pd.api.types.is_numeric_dtype(sat_df_full[c])
    ]
    if filename_prefix.startswith("LANDSAT8") or filename_prefix.startswith("Sentinel2"):
        nir_band_check, red_band_check = ('SR_B5', 'SR_B4') if filename_prefix.startswith("LANDSAT8") else ('B8', 'B4')
        if nir_band_check in sat_df_full.columns and red_band_check in sat_df_full.columns:
            if ndvi_base_name not in potential_base_feature_names: # Add if NDVI calculable
                potential_base_feature_names.append(ndvi_base_name)


    if filename_prefix.startswith("LANDSAT8") and 'QA_PIXEL' in sat_df_filtered_base.columns:
        qa_pixel_int = sat_df_filtered_base['QA_PIXEL'].astype(int)
        is_clear_or_water = ((qa_pixel_int & (1 << 6)) != 0) | ((qa_pixel_int & (1 << 7)) != 0)
        is_not_cloud_related_or_fill = (
            ((qa_pixel_int & (1 << 3)) == 0) & ((qa_pixel_int & (1 << 4)) == 0) &
            ((qa_pixel_int & (1 << 2)) == 0) & ((qa_pixel_int & (1 << 0)) == 0)   
        )
        good_pixel_mask_l8 = is_clear_or_water & is_not_cloud_related_or_fill
        sat_df_filtered_base = sat_df_filtered_base[good_pixel_mask_l8]

    if filename_prefix.startswith("Sentinel2") and 'SCL' in sat_df_filtered_base.columns:
        good_scl_values = [4, 5, 6, 7, 11] 
        scl_mask = sat_df_filtered_base['SCL'].isin(good_scl_values)
        sat_df_filtered_base = sat_df_filtered_base[scl_mask]

    # This placeholder creation needs to happen *after* potential_base_feature_names is defined
    # And it should use all_pids_from_base_str for the index
    def create_placeholder_df(pids_list, prefix, features_list, windows_cfg, k):
        final_placeholder = pd.DataFrame({'PID': pids_list})
        for win_name, _, _ in windows_cfg:
            for feat_base in features_list:
                final_placeholder[f"{prefix}_{feat_base}_knn_mean_k{k}_{win_name}"] = np.nan
        return final_placeholder

    if sat_df_filtered_base.empty:
        # print(f"No data remaining for {filename_prefix} after QA masking.")
        return create_placeholder_df(all_pids_from_base_str, filename_prefix, potential_base_feature_names, time_windows_config, k_neighbors)


    if filename_prefix.startswith("LANDSAT8") or filename_prefix.startswith("Sentinel2"):
        nir_band, red_band = ('SR_B5', 'SR_B4') if filename_prefix.startswith("LANDSAT8") else ('B8', 'B4')
        if nir_band in sat_df_filtered_base.columns and red_band in sat_df_filtered_base.columns:
            if sat_df_filtered_base[nir_band].notna().any() and sat_df_filtered_base[red_band].notna().any():
                numerator = sat_df_filtered_base[nir_band].astype(float) - sat_df_filtered_base[red_band].astype(float)
                denominator = (sat_df_filtered_base[nir_band].astype(float) + sat_df_filtered_base[red_band].astype(float)).replace(0, np.nan)
                sat_df_filtered_base[ndvi_base_name] = numerator / denominator
                sat_df_filtered_base[ndvi_base_name].fillna(0, inplace=True)
            else:
                 sat_df_filtered_base[ndvi_base_name] = np.nan
        elif ndvi_base_name in potential_base_feature_names: # If NDVI was expected but bands missing post-QA
            sat_df_filtered_base[ndvi_base_name] = np.nan


    feature_cols_sat_to_process = [
        c for c in sat_df_filtered_base.columns 
        if pd.api.types.is_numeric_dtype(sat_df_filtered_base[c]) and 
           c not in ['PID', date_col_sat, 'QA_PIXEL', 'SCL']
    ]
    if ndvi_base_name in sat_df_filtered_base.columns and ndvi_base_name not in feature_cols_sat_to_process:
        feature_cols_sat_to_process.append(ndvi_base_name)

    if not feature_cols_sat_to_process:
        # print(f"No numeric features to process for {filename_prefix} after QA/NDVI.")
        return create_placeholder_df(all_pids_from_base_str, filename_prefix, potential_base_feature_names, time_windows_config, k_neighbors)

    sat_df_filtered_base.sort_values(by=['PID', date_col_sat], inplace=True)
    all_window_features_dfs = []
    reference_end_date_main = pd.to_datetime(reference_end_date_main_str)

    for window_name, duration_days, offset_days in time_windows_config:
        start_date = reference_end_date_main - timedelta(days=offset_days + duration_days)
        end_date = reference_end_date_main - timedelta(days=offset_days)
        window_data = sat_df_filtered_base[
            (sat_df_filtered_base[date_col_sat] >= start_date) & 
            (sat_df_filtered_base[date_col_sat] < end_date)
        ].copy()

        if window_data.empty:
            # Create a DF with NaNs for all PIDs for this window's features
            agg_data_for_window = pd.DataFrame(index=pd.Index(all_pids_from_base_str, name='PID'))
            for fc_base in feature_cols_sat_to_process: # Use actual processable features
                agg_data_for_window[f"{filename_prefix}_{fc_base}_knn_mean_k{k_neighbors}_{window_name}"] = np.nan
        else:
            agg_data_for_window = window_data.groupby('PID')[feature_cols_sat_to_process].apply(
                lambda x: x.tail(k_neighbors).mean(skipna=True) 
            )
            agg_data_for_window.columns = [
                f"{filename_prefix}_{c_base}_knn_mean_k{k_neighbors}_{window_name}" 
                for c_base in agg_data_for_window.columns
            ]
            agg_data_for_window = agg_data_for_window.reindex(all_pids_from_base_str) # Ensure all PIDs
        
        all_window_features_dfs.append(agg_data_for_window.reset_index())

    final_sat_df = pd.DataFrame({'PID': all_pids_from_base_str}) # Start with all PIDs as strings
    for df_w in all_window_features_dfs:
        if not df_w.empty and 'PID' in df_w.columns:
            # Ensure df_w['PID'] is string if not already, for robust merge
            df_w['PID'] = df_w['PID'].astype(str)
            cols_to_merge = [c for c in df_w.columns if c != 'PID']
            if cols_to_merge:
                 final_sat_df = pd.merge(final_sat_df, df_w[['PID'] + cols_to_merge], on='PID', how='left')
    
    del sat_df_full, sat_df_filtered_base, window_data; gc.collect()
    return final_sat_df


# --- 1. Load Core Data ---
print("Loading core data...")
train_df_orig = pd.read_csv(f'{BASE_PATH}Train.csv')
test_df_orig = pd.read_csv(f'{BASE_PATH}Test.csv')

# Ensure PIDs are strings from the beginning
train_df_orig['PID'] = train_df_orig['PID'].astype(str)
test_df_orig['PID'] = test_df_orig['PID'].astype(str)


if 'BD' in train_df_orig.columns: train_df_orig.rename(columns={'BD': 'BulkDensity'}, inplace=True)
if 'BD' in test_df_orig.columns: test_df_orig.rename(columns={'BD': 'BulkDensity'}, inplace=True)

# --- 2. Combine Train and Test & Initial Preprocessing ---
train_len = len(train_df_orig); train_df_orig['is_train'] = 1; test_df_orig['is_train'] = 0
for nut in TARGET_NUTRIENTS:
    if nut not in train_df_orig.columns: train_df_orig[nut] = np.nan
    if nut not in test_df_orig.columns: test_df_orig[nut] = np.nan

# original_test_bd_df PIDs will be strings
original_test_bd_df = test_df_orig[['PID', 'BulkDensity']].copy()
original_test_bd_df.rename(columns={'BulkDensity': 'BulkDensity_original'}, inplace=True)
if original_test_bd_df['BulkDensity_original'].isnull().any():
    bd_train_median_orig = train_df_orig['BulkDensity'].median(skipna=True)
    original_test_bd_df['BulkDensity_original'].fillna(bd_train_median_orig if not pd.isna(bd_train_median_orig) else 1.2, inplace=True)

combined_df = pd.concat([train_df_orig, test_df_orig], axis=0, ignore_index=True)
# all_pids_in_dataset will be strings
all_pids_in_dataset = combined_df['PID'].unique().tolist()


del train_df_orig, test_df_orig; gc.collect()

# --- 2.5 Add Missing Indicator Flags & Outlier Capping ---
print("\n--- Adding Missing Indicator Flags & Capping Outliers ---")
core_features_to_flag_missing = ['ecec20', 'hp20', 'xhp20', 'BulkDensity']
for col in core_features_to_flag_missing:
    if col in combined_df.columns and combined_df[col].isnull().any(): combined_df[f'{col}_is_missing'] = combined_df[col].isnull().astype(int)
    else: combined_df[f'{col}_is_missing'] = 0
features_for_capping_present = [f for f in FEATURES_FOR_OUTLIER_CAPPING if f in combined_df.columns]
for col in features_for_capping_present:
    if combined_df[col].isnull().all(): continue
    lower_cap = combined_df.loc[combined_df['is_train'] == 1, col].quantile(LOWER_PERCENTILE_CAP)
    upper_cap = combined_df.loc[combined_df['is_train'] == 1, col].quantile(UPPER_PERCENTILE_CAP)
    if pd.isna(lower_cap) or pd.isna(upper_cap) or lower_cap >= upper_cap: continue
    combined_df[col] = np.clip(combined_df[col], lower_cap, upper_cap)


# --- 3. Satellite Data Integration ---
print(f"\n--- Starting Satellite Data Integration ---")
satellite_files_config = [
    (f'{BASE_PATH}LANDSAT8_data_updated.csv', 'date'), (f'{BASE_PATH}MODIS_MOD16A2_data.csv', 'date'),
    (f'{BASE_PATH}MODIS_MCD43A4_data.csv', 'date'), (f'{BASE_PATH}MODIS_MOD09GA_data.csv', 'date'),
    (f'{BASE_PATH}Sentinel2_data.csv', 'date'), (f'{BASE_PATH}Sentinel1_data.csv', 'date'),
    (f'{BASE_PATH}MODIS_MOD13Q1_data.csv', 'date'), (f'{BASE_PATH}MODIS_MOD11A1_data.csv', 'date')
]
for sat_file_path, _ in satellite_files_config:
    print(f"Processing satellite file: {sat_file_path.split('/')[-1]}")
    # all_pids_in_dataset are already strings
    processed_sat_df = load_and_preprocess_satellite_data_multi_window_knn_features(
        filepath=sat_file_path, all_pids_from_base=all_pids_in_dataset,
        reference_end_date_main_str=REFERENCE_END_DATE_OF_MAIN_WINDOW_STR,
        time_windows_config=SATELLITE_TIME_WINDOWS, k_neighbors=K_FOR_KNN_FEATURES
    )
    if 'PID' in processed_sat_df.columns and not processed_sat_df.drop(columns=['PID'], errors='ignore').empty:
        # Ensure PID in processed_sat_df is also string for merging with combined_df['PID'] (which is string)
        processed_sat_df['PID'] = processed_sat_df['PID'].astype(str)
        cols_before_merge = set(combined_df.columns)
        combined_df = pd.merge(combined_df, processed_sat_df, on='PID', how='left') # Both PIDs are string
        cols_after_merge = set(combined_df.columns)
        newly_added_agg_cols = list(cols_after_merge - cols_before_merge)
        for agg_col in newly_added_agg_cols:
            if agg_col in combined_df.columns and combined_df[agg_col].isnull().any():
                combined_df[f'{agg_col}_is_missing'] = combined_df[agg_col].isnull().astype(int)
            else: combined_df[f'{agg_col}_is_missing'] = 0
    del processed_sat_df; gc.collect()
print("\nFinished satellite data integration.")


# --- 4. Feature Engineering (Polynomials, Interactions) ---
print("\n--- Advanced Static and EO Interaction Feature Engineering ---")
key_static_features_for_fe = ['pH', 'soc20', 'cec20', 'BulkDensity', 'alb', 'slope']
for col in key_static_features_for_fe:
    if col in combined_df.columns and combined_df[col].isnull().any():
        median_train = combined_df.loc[combined_df['is_train'] == 1, col].median()
        combined_df[col].fillna(median_train if not pd.isna(median_train) else 0, inplace=True)
poly_individual_candidates = ['pH', 'soc20', 'cec20', 'BulkDensity']
poly_individual_to_use = [f for f in poly_individual_candidates if f in combined_df.columns]
if poly_individual_to_use:
    poly_creator_individual = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
    try:
        source_df_for_poly_individual = combined_df[poly_individual_to_use].copy()
        for col_p in poly_individual_to_use:
            if source_df_for_poly_individual[col_p].isnull().any():
                 median_val = source_df_for_poly_individual[col_p].median()
                 source_df_for_poly_individual[col_p].fillna(median_val if not pd.isna(median_val) else 0, inplace=True)
        poly_arr_individual = poly_creator_individual.fit_transform(source_df_for_poly_individual)
        poly_feature_names_individual = poly_creator_individual.get_feature_names_out(input_features=poly_individual_to_use)
        poly_df_individual = pd.DataFrame(poly_arr_individual, columns=poly_feature_names_individual, index=combined_df.index)
        new_terms_from_poly_individual = [name for name in poly_feature_names_individual if name not in poly_individual_to_use and name not in combined_df.columns]
        if new_terms_from_poly_individual: combined_df = pd.concat([combined_df, poly_df_individual[new_terms_from_poly_individual]], axis=1)
    except Exception as e_poly_ind: print(f"  Error creating individual polynomial features: {e_poly_ind}")

static_interaction_candidates = ['pH', 'soc20', 'cec20', 'BulkDensity', 'alb', 'slope']
for col1, col2 in itertools.combinations(static_interaction_candidates, 2):
    if col1 in combined_df.columns and col2 in combined_df.columns:
        for c_int in [col1, col2]:
            if combined_df[c_int].isnull().any():
                median_val_int_train = combined_df.loc[combined_df['is_train'] == 1, c_int].median()
                combined_df[c_int].fillna(median_val_int_train if not pd.isna(median_val_int_train) else 0, inplace=True)
        interaction_col_name = f'{col1}_x_{col2}'
        if interaction_col_name not in combined_df.columns: combined_df[interaction_col_name] = combined_df[col1] * combined_df[col2]

example_eo_ndvi_feature = f'LANDSAT8_NDVI_calc_knn_mean_k{K_FOR_KNN_FEATURES}_win90d_mid'
static_feat_for_eo_interaction = 'pH'
if static_feat_for_eo_interaction in combined_df.columns and example_eo_ndvi_feature in combined_df.columns:
    if combined_df[static_feat_for_eo_interaction].isnull().any():
        median_static = combined_df.loc[combined_df['is_train']==1, static_feat_for_eo_interaction].median(skipna=True)
        combined_df[static_feat_for_eo_interaction].fillna(median_static if not pd.isna(median_static) else 0, inplace=True)
    if combined_df[example_eo_ndvi_feature].isnull().any():
        median_eo = combined_df.loc[combined_df['is_train']==1, example_eo_ndvi_feature].median(skipna=True)
        combined_df[example_eo_ndvi_feature].fillna(median_eo if not pd.isna(median_eo) else 0, inplace=True)
    interaction_name_static_eo = f'{static_feat_for_eo_interaction}_x_LS8NDVI90d'
    if interaction_name_static_eo not in combined_df.columns:
        combined_df[interaction_name_static_eo] = combined_df[static_feat_for_eo_interaction] * combined_df[example_eo_ndvi_feature]


# --- 5. Final Preprocessing (Label Encoding, Final Imputation) ---
print("\n--- Final Preprocessing ---")
# PID is already string, so it won't be in categorical_cols if we remove it based on name
categorical_cols = combined_df.select_dtypes(include='object').columns.tolist()
if 'PID' in categorical_cols: categorical_cols.remove('PID') # PID is string, but we don't label encode it
if 'site' in combined_df.columns and 'site' in categorical_cols: categorical_cols.remove('site')
for col in categorical_cols:
    combined_df[col] = combined_df[col].fillna('Missing_Cat').astype(str)
    le = LabelEncoder(); combined_df[col] = le.fit_transform(combined_df[col])

# PID is string, so it's not numeric and won't be in features_for_model
features_for_model = [f for f in combined_df.columns if f not in TARGET_NUTRIENTS + ['PID', 'is_train', 'site'] and pd.api.types.is_numeric_dtype(combined_df[f])]
print("  Performing final median imputation...")
for col in features_for_model:
    if combined_df[col].isnull().any():
        median_val_to_use = combined_df.loc[combined_df['is_train'] == 1, col].median()
        if pd.isna(median_val_to_use): median_val_to_use = combined_df[col].median()
        if pd.isna(median_val_to_use): median_val_to_use = 0
        combined_df[col].fillna(median_val_to_use, inplace=True)
cols_to_drop_final_check = []
for f in features_for_model:
    if combined_df[f].isnull().all(): cols_to_drop_final_check.append(f)
    elif combined_df.loc[combined_df['is_train']==1, f].nunique(dropna=False) <= 1 and combined_df.loc[combined_df['is_train']==1, f].notnull().any():
        cols_to_drop_final_check.append(f)
if cols_to_drop_final_check:
    print(f"  Dropping {len(cols_to_drop_final_check)} features (all NaN or no variance in train): {cols_to_drop_final_check}")
    features_for_model = [f for f in features_for_model if f not in cols_to_drop_final_check]
    combined_df.drop(columns=cols_to_drop_final_check, inplace=True)
print(f"Number of features for modeling: {len(features_for_model)}")


# --- 6. Prepare Data for Models (Train/Test Split, Scaling for GLM) ---
train_processed_unscaled = combined_df.loc[combined_df['is_train'] == 1, features_for_model + TARGET_NUTRIENTS].copy()
# IMPORTANT: Keep PID (which is string) in test_processed_unscaled
test_processed_unscaled = combined_df.loc[combined_df['is_train'] == 0, features_for_model + ['PID']].copy()

# Data for LGBM (uses unscaled features)
X_train_lgbm_full = train_processed_unscaled[features_for_model]
y_train_lgbm_full = train_processed_unscaled[TARGET_NUTRIENTS]
X_test_lgbm = test_processed_unscaled[features_for_model] # This is X_test_orig_df for pseudo-labeling

# Data for GLM (needs scaling)
print("  Scaling features for GLM using StandardScaler...")
scaler = StandardScaler()
X_train_glm_scaled_vals = scaler.fit_transform(X_train_lgbm_full)
X_test_glm_scaled_vals = scaler.transform(X_test_lgbm)

X_train_glm_scaled = pd.DataFrame(X_train_glm_scaled_vals, columns=features_for_model, index=X_train_lgbm_full.index)
X_test_glm_scaled = pd.DataFrame(X_test_glm_scaled_vals, columns=features_for_model, index=X_test_lgbm.index)

del combined_df; gc.collect()


# --- 7. Model Training ---
# PIDs are sourced from test_processed_unscaled['PID'] which are strings
all_test_pids_final = test_processed_unscaled['PID'].astype(str).values # Already string, but enforce

test_predictions_lgbm_ppm = pd.DataFrame({'PID': all_test_pids_final})
test_predictions_glm_ppm = pd.DataFrame({'PID': all_test_pids_final})

# 7.A LightGBM Models
print("\n--- Training LightGBM Models ---")
lgbm_cpu_params = {
    'objective': 'rmse', 'metric': 'rmse', 'n_estimators': 1000, 'learning_rate': 0.02,
    'feature_fraction': 0.7, 'bagging_fraction': 0.7, 'bagging_freq': 1,
    'lambda_l1': 0.1, 'lambda_l2': 0.1, 'num_leaves': 41, 'max_depth': -1,
    'min_child_samples': 20, 'verbose': -1, 'seed': SEED, 'boosting_type': 'gbdt', 'n_jobs': -1
}
for nutrient in TARGET_NUTRIENTS:
    print(f"  Training LGBM for Nutrient: {nutrient}")
    y_train_nutrient = y_train_lgbm_full[nutrient].copy()
    valid_target_idx = y_train_nutrient.dropna().index

    if len(valid_target_idx) == 0:
        print(f"    Skipping LGBM for {nutrient} due to no valid training targets.")
        test_predictions_lgbm_ppm[nutrient] = 0
        continue

    y_train_final_nutrient = y_train_nutrient.loc[valid_target_idx]
    X_train_final_nutrient_lgbm = X_train_lgbm_full.loc[valid_target_idx]
    selected_features_lgbm = features_for_model

    if X_train_final_nutrient_lgbm[selected_features_lgbm].empty:
        print(f"    Skipping LGBM for {nutrient} due to empty X_train after filtering.")
        test_predictions_lgbm_ppm[nutrient] = 0
        continue

    try:
        if ENABLE_PSEUDO_LABELING_LGBM:
            print(f"    Applying confident pseudo-labeling for LGBM on {nutrient}...")
            # X_test_lgbm's index is important for pseudo-labeling to map back predictions.
            # Its index comes from test_processed_unscaled.
            model_lgbm = confident_pseudo_labeling_lgbm( # Using the simpler PL function
                initial_model_params=lgbm_cpu_params.copy(),
                X_train_orig_df=X_train_final_nutrient_lgbm.copy(),
                y_train_orig_series=y_train_final_nutrient.copy(),
                X_test_orig_df=X_test_lgbm.copy(), # Pass features of test set
                selected_features_list=selected_features_lgbm,
                iterations=PSEUDO_LABELING_ITERATIONS,
                confidence_percentile=PSEUDO_LABELING_CONFIDENCE_PERCENTILE,
                n_bootstrap_for_confidence=PSEUDO_LABELING_N_BOOTSTRAP_CONF,
                base_seed=SEED,
                min_confident_samples_to_add=PSEUDO_LABELING_MIN_SAMPLES
            )
        else:
            model_lgbm = lgb.LGBMRegressor(**lgbm_cpu_params)
            model_lgbm.fit(X_train_final_nutrient_lgbm[selected_features_lgbm], y_train_final_nutrient)

        preds_lgbm = model_lgbm.predict(X_test_lgbm[selected_features_lgbm])
        test_predictions_lgbm_ppm[nutrient] = np.maximum(0, preds_lgbm)
    except Exception as e:
        print(f"    Error during LGBM training for {nutrient} (pseudo-labeling: {ENABLE_PSEUDO_LABELING_LGBM}): {e}")
        import traceback
        traceback.print_exc()
        test_predictions_lgbm_ppm[nutrient] = 0

# 7.B GLM (TweedieRegressor) Models
print("\n--- Training GLM (TweedieRegressor) Models ---")
glm_alpha = 0.1; glm_power = 2
for nutrient in TARGET_NUTRIENTS:
    print(f"  Training GLM for Nutrient: {nutrient}")
    y_train_nutrient_glm = y_train_lgbm_full[nutrient].copy()
    y_train_nutrient_glm = np.maximum(y_train_nutrient_glm, 1e-6 if glm_power > 0 else -np.inf)
    valid_target_idx_glm = y_train_nutrient_glm.dropna().index

    if len(valid_target_idx_glm) == 0:
        print(f"    Skipping GLM for {nutrient} due to no valid training targets.")
        test_predictions_glm_ppm[nutrient] = 0
        continue

    y_train_final_glm = y_train_nutrient_glm.loc[valid_target_idx_glm]
    X_train_final_glm_scaled = X_train_glm_scaled.loc[valid_target_idx_glm]

    if X_train_final_glm_scaled.empty:
        print(f"    Skipping GLM for {nutrient} due to empty X_train after filtering.")
        test_predictions_glm_ppm[nutrient] = 0
        continue

    model_glm = TweedieRegressor(power=glm_power, alpha=glm_alpha, link='log', max_iter=1000, tol=1e-4)
    try:
        model_glm.fit(X_train_final_glm_scaled[features_for_model], y_train_final_glm)
        preds_glm = model_glm.predict(X_test_glm_scaled[features_for_model])
        test_predictions_glm_ppm[nutrient] = np.maximum(0, preds_glm)
    except Exception as e:
        print(f"    Error GLM {nutrient}: {e}")
        test_predictions_glm_ppm[nutrient] = 0

# --- 8. Ensemble Predictions & Generate Submission ---
print("\n--- Ensembling Predictions and Generating Submission ---")

# PIDs in test_predictions_..._ppm are already string.
# PIDs in original_test_bd_df are already string (set in Sec 1).
original_test_bd_df['PID'] = original_test_bd_df['PID'].astype(str) # Re-ensure for safety

test_predictions_ensembled_ppm = test_predictions_lgbm_ppm[['PID']].copy()

for nutrient in TARGET_NUTRIENTS:
    lgbm_preds_series = test_predictions_lgbm_ppm.get(nutrient)
    glm_preds_series = test_predictions_glm_ppm.get(nutrient)

    lgbm_preds = lgbm_preds_series.values if lgbm_preds_series is not None else np.zeros(len(test_predictions_ensembled_ppm))
    if lgbm_preds_series is None: print(f"Warning: LGBM preds missing for {nutrient}")
        
    glm_preds = glm_preds_series.values if glm_preds_series is not None else np.zeros(len(test_predictions_ensembled_ppm))
    if glm_preds_series is None: print(f"Warning: GLM preds missing for {nutrient}")
        
    test_predictions_ensembled_ppm[nutrient] = (WEIGHT_LGBM * lgbm_preds) + (WEIGHT_GLM * glm_preds)

test_pred_ppm_long = test_predictions_ensembled_ppm.melt(
    id_vars=['PID'], value_vars=TARGET_NUTRIENTS, var_name='Nutrient', value_name='Predicted_PPM')
# test_pred_ppm_long['PID'] is string

print(f"Debug section 8 - PIDs dtypes before merge 1:\n"
      f"  test_pred_ppm_long['PID']: {test_pred_ppm_long['PID'].dtype}\n"
      f"  original_test_bd_df['PID']: {original_test_bd_df['PID'].dtype}")

final_submission_prep_df = pd.merge(test_pred_ppm_long, original_test_bd_df, on='PID', how='left')
bulk_density_col_for_gap = 'BulkDensity_original'

try:
    gap_test_df_orig = pd.read_csv(f'{BASE_PATH}Gap_Test.csv')
    gap_test_df_orig['PID'] = gap_test_df_orig['PID'].astype(str)
    gap_test_df_orig['Nutrient'] = gap_test_df_orig['Nutrient'].astype(str)
    
    final_submission_prep_df['Nutrient'] = final_submission_prep_df['Nutrient'].astype(str)
    
    print(f"Debug section 8 - PIDs/Nutrient dtypes before merge 2:\n"
          f"  final_submission_prep_df['PID']: {final_submission_prep_df['PID'].dtype}\n"
          f"  final_submission_prep_df['Nutrient']: {final_submission_prep_df['Nutrient'].dtype}\n"
          f"  gap_test_df_orig['PID']: {gap_test_df_orig['PID'].dtype}\n"
          f"  gap_test_df_orig['Nutrient']: {gap_test_df_orig['Nutrient'].dtype}")

    final_submission_prep_df = pd.merge(final_submission_prep_df, gap_test_df_orig[['PID', 'Nutrient', 'Required']], on=['PID', 'Nutrient'], how='left')
except FileNotFoundError:
    print("Warning: Gap_Test.csv not found. 'Required' column will be set to 0.")
    final_submission_prep_df['Required'] = 0
except Exception as e_gap:
    print(f"Error loading or merging Gap_Test.csv: {e_gap}. 'Required' column will be set to 0.")
    if 'Required' not in final_submission_prep_df.columns: final_submission_prep_df['Required'] = 0

if bulk_density_col_for_gap not in final_submission_prep_df.columns:
    final_submission_prep_df[bulk_density_col_for_gap] = 1.2
final_submission_prep_df[bulk_density_col_for_gap].fillna(1.2, inplace=True)

final_submission_prep_df['Predicted_PPM'].fillna(0, inplace=True)

if 'Required' not in final_submission_prep_df.columns:
    final_submission_prep_df['Required'] = 0
final_submission_prep_df['Required'].fillna(0, inplace=True)


final_submission_prep_df['Available_kg_ha'] = final_submission_prep_df['Predicted_PPM'] * SOIL_DEPTH_CM * final_submission_prep_df[bulk_density_col_for_gap] * 0.1
final_submission_prep_df['Gap'] = final_submission_prep_df['Required'] - final_submission_prep_df['Available_kg_ha']
final_submission_prep_df['Gap'].fillna(0, inplace=True)

final_submission_prep_df['ID'] = final_submission_prep_df['PID'].astype(str) + '_' + final_submission_prep_df['Nutrient'].astype(str)
submission_df_final = final_submission_prep_df[['ID', 'Gap']].copy() # Use .copy()

# Final check for missing IDs
num_expected_rows = len(test_processed_unscaled['PID'].unique()) * len(TARGET_NUTRIENTS)
if len(submission_df_final) != num_expected_rows:
    print(f"WARNING: Row count mismatch! Expected {num_expected_rows}, Got {len(submission_df_final)}")
    # Create a DataFrame of all expected IDs
    expected_pids_final = test_processed_unscaled['PID'].astype(str).unique()
    expected_ids_df = pd.DataFrame(list(itertools.product(expected_pids_final, TARGET_NUTRIENTS)), columns=['PID', 'Nutrient'])
    expected_ids_df['ID_expected'] = expected_ids_df['PID'] + '_' + expected_ids_df['Nutrient']
    
    missing_in_submission = set(expected_ids_df['ID_expected']) - set(submission_df_final['ID'])
    if missing_in_submission:
        print(f"IDs expected but missing in final submission (first 10): {list(missing_in_submission)[:10]}")
        if any('ID_ZgrspC' in mid for mid in missing_in_submission):
             print("One of the problematic 'ID_ZgrspC' IDs is missing.")


submission_filename = f'submission_full_v_simpler_pl_robust_pid.csv'
submission_df_final.to_csv(submission_filename, index=False)
print(f"\nSubmission file '{submission_filename}' created successfully.")
print(f"Number of rows in submission: {len(submission_df_final)}")
print(submission_df_final.head())
print("\nDone!")

Loading core data...

--- Adding Missing Indicator Flags & Capping Outliers ---

--- Starting Satellite Data Integration ---
Processing satellite file: LANDSAT8_data_updated.csv
Processing satellite file: MODIS_MOD16A2_data.csv
Processing satellite file: MODIS_MCD43A4_data.csv
Processing satellite file: MODIS_MOD09GA_data.csv
Processing satellite file: Sentinel2_data.csv
Processing satellite file: Sentinel1_data.csv
Processing satellite file: MODIS_MOD13Q1_data.csv
Processing satellite file: MODIS_MOD11A1_data.csv

Finished satellite data integration.

--- Advanced Static and EO Interaction Feature Engineering ---

--- Final Preprocessing ---
  Performing final median imputation...
  Dropping 64 features (all NaN or no variance in train): ['LANDSAT8_QA_RADSAT_knn_mean_k3_win30d_recent', 'LANDSAT8_QA_RADSAT_knn_mean_k3_win90d_mid', 'LANDSAT8_QA_RADSAT_knn_mean_k3_win180d_early', 'LANDSAT8_QA_RADSAT_knn_mean_k3_win_full_year', 'Sentinel2_B1_knn_mean_k3_win30d_recent', 'Sentinel2_B11_knn_

In [6]:
import xgboost as xgb # For XGBoost

In [ ]:
# --- 7.C XGBoost Models ---
print("\n--- Training XGBoost Models ---")
test_predictions_xgb_ppm = pd.DataFrame({'PID': test_processed_unscaled['PID'].values})

# Define XGBoost parameters
# Note: 'early_stopping_rounds' is removed from here.
# If you want to use it, you must provide an 'eval_set' to the fit method.
xgb_constructor_params = {
    'objective': 'reg:squarederror',  # For regression
    'eval_metric': 'rmse',            # Evaluation metric
    'eta': 0.02,                      # Learning rate
    'max_depth': 7,                   # Maximum depth of a tree
    'subsample': 0.8,                 # Subsample ratio of the training instance
    'colsample_bytree': 0.8,          # Subsample ratio of columns when constructing each tree
    'lambda': 1,                      # L2 regularization term on weights (xgb's lambda)
    'alpha': 0.1,                     # L1 regularization term on weights (xgb's alpha)
    'seed': SEED,
    'n_estimators': 1000,             # Number of boosting rounds
    'n_jobs': -1                      # Use all available CPU cores
}
# If you want to use early stopping, define it separately
# XGB_EARLY_STOPPING_ROUNDS = 50

for nutrient in TARGET_NUTRIENTS:
    print(f"  Training XGBoost for Nutrient: {nutrient}")
    y_train_nutrient_xgb = y_train_lgbm_full[nutrient].copy()
    valid_target_idx_xgb = y_train_nutrient_xgb.dropna().index

    if len(valid_target_idx_xgb) == 0:
        print(f"    Skipping XGBoost for {nutrient} due to no valid training targets.")
        test_predictions_xgb_ppm[nutrient] = 0
        continue

    y_train_final_nutrient_xgb = y_train_nutrient_xgb.loc[valid_target_idx_xgb]
    X_train_final_nutrient_xgb = X_train_lgbm_full.loc[valid_target_idx_xgb, features_for_model]

    if X_train_final_nutrient_xgb.empty:
        print(f"    Skipping XGBoost for {nutrient} due to empty X_train after filtering.")
        test_predictions_xgb_ppm[nutrient] = 0
        continue

    try:

        model_xgb = xgb.XGBRegressor(**xgb_constructor_params)


        model_xgb.fit(X_train_final_nutrient_xgb, y_train_final_nutrient_xgb, verbose=False)


        preds_xgb = model_xgb.predict(X_test_lgbm[features_for_model])
        test_predictions_xgb_ppm[nutrient] = np.maximum(0, preds_xgb)
    except Exception as e:
        print(f"    Error during XGBoost training for {nutrient}: {e}")
        test_predictions_xgb_ppm[nutrient] = 0
    gc.collect()


--- Training XGBoost Models ---
  Training XGBoost for Nutrient: N
  Training XGBoost for Nutrient: P
  Training XGBoost for Nutrient: K
  Training XGBoost for Nutrient: Ca
  Training XGBoost for Nutrient: Mg
  Training XGBoost for Nutrient: S
  Training XGBoost for Nutrient: Zn
  Training XGBoost for Nutrient: Mn
  Training XGBoost for Nutrient: Fe
  Training XGBoost for Nutrient: Cu
  Training XGBoost for Nutrient: B


In [ ]:
# Ensemble Weights
WEIGHT_LGBM = 0.25  
WEIGHT_GLM = 0.25   
WEIGHT_XGB = 0.5   

In [ ]:

# --- 8. Ensemble Predictions & Generate Submission ---
print("\n--- Ensembling Predictions and Generating Submission (LGBM + GLM + XGB) ---")

# PIDs in test_predictions_..._ppm are already string (from Section 7 init).
# PIDs in original_test_bd_df are already string (set in Sec 1).
original_test_bd_df['PID'] = original_test_bd_df['PID'].astype(str) # Re-ensure for safety

# Create ensembled DataFrame starting with PIDs from one of the prediction DFs
# All prediction DFs (LGBM, GLM, XGB) should have the same PIDs in the same order
test_predictions_ensembled_ppm = test_predictions_lgbm_ppm[['PID']].copy() # Has string PIDs

for nutrient in TARGET_NUTRIENTS:
    lgbm_preds_series = test_predictions_lgbm_ppm.get(nutrient)
    glm_preds_series = test_predictions_glm_ppm.get(nutrient)
    xgb_preds_series = test_predictions_xgb_ppm.get(nutrient) # Get XGBoost predictions

    # Use .values to ensure alignment if Series come from different sources
    # Default to an array of zeros if a prediction series is somehow missing
    # (shouldn't happen if columns were initialized with 0 for failed nutrients)
    len_preds = len(test_predictions_ensembled_ppm) # Number of test samples

    lgbm_preds = lgbm_preds_series.values if lgbm_preds_series is not None else np.zeros(len_preds)
    if lgbm_preds_series is None: print(f"Warning: LGBM preds series missing for {nutrient}")
        
    glm_preds = glm_preds_series.values if glm_preds_series is not None else np.zeros(len_preds)
    if glm_preds_series is None: print(f"Warning: GLM preds series missing for {nutrient}")

    xgb_preds = xgb_preds_series.values if xgb_preds_series is not None else np.zeros(len_preds)
    if xgb_preds_series is None: print(f"Warning: XGB preds series missing for {nutrient}")
        
    # Ensemble with XGBoost
    test_predictions_ensembled_ppm[nutrient] = (
        WEIGHT_LGBM * lgbm_preds +
        WEIGHT_GLM * glm_preds +
        WEIGHT_XGB * xgb_preds
    )

test_pred_ppm_long = test_predictions_ensembled_ppm.melt(
    id_vars=['PID'], value_vars=TARGET_NUTRIENTS, var_name='Nutrient', value_name='Predicted_PPM')
# test_pred_ppm_long['PID'] is string

print(f"Debug section 8 - PIDs dtypes before merge 1:\n"
      f"  test_pred_ppm_long['PID']: {test_pred_ppm_long['PID'].dtype}\n"
      f"  original_test_bd_df['PID']: {original_test_bd_df['PID'].dtype}")

final_submission_prep_df = pd.merge(test_pred_ppm_long, original_test_bd_df, on='PID', how='left')
bulk_density_col_for_gap = 'BulkDensity_original' 

try:
    gap_test_df_orig = pd.read_csv(f'{BASE_PATH}Gap_Test.csv')
    gap_test_df_orig['PID'] = gap_test_df_orig['PID'].astype(str) # Ensure string PID
    gap_test_df_orig['Nutrient'] = gap_test_df_orig['Nutrient'].astype(str) # Ensure string Nutrient
    
    # Ensure Nutrient in final_submission_prep_df is also string for the merge
    final_submission_prep_df['Nutrient'] = final_submission_prep_df['Nutrient'].astype(str)
    
    print(f"Debug section 8 - PIDs/Nutrient dtypes before merge 2:\n"
          f"  final_submission_prep_df['PID']: {final_submission_prep_df['PID'].dtype}\n"
          f"  final_submission_prep_df['Nutrient']: {final_submission_prep_df['Nutrient'].dtype}\n"
          f"  gap_test_df_orig['PID']: {gap_test_df_orig['PID'].dtype}\n"
          f"  gap_test_df_orig['Nutrient']: {gap_test_df_orig['Nutrient'].dtype}")

    final_submission_prep_df = pd.merge(final_submission_prep_df, gap_test_df_orig[['PID', 'Nutrient', 'Required']], on=['PID', 'Nutrient'], how='left')
except FileNotFoundError:
    print("Warning: Gap_Test.csv not found. 'Required' column will be set to 0.")
    final_submission_prep_df['Required'] = 0 # Create column if not exists
except Exception as e_gap:
    print(f"Error loading or merging Gap_Test.csv: {e_gap}. 'Required' column will be set to 0.")
    if 'Required' not in final_submission_prep_df.columns: final_submission_prep_df['Required'] = 0

# NaN Filling (as per your working code's logic, simplified)
if bulk_density_col_for_gap not in final_submission_prep_df.columns: # Should exist due to merge
    final_submission_prep_df[bulk_density_col_for_gap] = 1.2
final_submission_prep_df[bulk_density_col_for_gap].fillna(1.2, inplace=True)

# Predicted_PPM can be NaN if a model failed for a nutrient. Fill with 0.
final_submission_prep_df['Predicted_PPM'].fillna(0, inplace=True)

# Required can be NaN if a PID-Nutrient pair from test set isn't in Gap_Test.csv. Fill with 0.
if 'Required' not in final_submission_prep_df.columns: # If Gap_Test.csv failed to load/merge
    final_submission_prep_df['Required'] = 0
final_submission_prep_df['Required'].fillna(0, inplace=True)


final_submission_prep_df['Available_kg_ha'] = final_submission_prep_df['Predicted_PPM'] * SOIL_DEPTH_CM * final_submission_prep_df[bulk_density_col_for_gap] * 0.1
final_submission_prep_df['Gap'] = final_submission_prep_df['Required'] - final_submission_prep_df['Available_kg_ha']
# Ensure Gap itself is not NaN
final_submission_prep_df['Gap'].fillna(0, inplace=True)


final_submission_prep_df['ID'] = final_submission_prep_df['PID'].astype(str) + '_' + final_submission_prep_df['Nutrient'].astype(str)
submission_df_final = final_submission_prep_df[['ID', 'Gap']].copy() # Use .copy()

# Final check for missing IDs
# Assuming test_processed_unscaled is available from Section 6
num_expected_rows = len(test_processed_unscaled['PID'].unique()) * len(TARGET_NUTRIENTS)
if len(submission_df_final) != num_expected_rows:
    print(f"WARNING: Row count mismatch! Expected {num_expected_rows}, Got {len(submission_df_final)}")
    # Create a DataFrame of all expected IDs
    expected_pids_final = test_processed_unscaled['PID'].astype(str).unique()
    expected_ids_df = pd.DataFrame(list(itertools.product(expected_pids_final, TARGET_NUTRIENTS)), columns=['PID', 'Nutrient'])
    expected_ids_df['ID_expected'] = expected_ids_df['PID'] + '_' + expected_ids_df['Nutrient']
    
    missing_in_submission = set(expected_ids_df['ID_expected']) - set(submission_df_final['ID'])
    if missing_in_submission:
        print(f"IDs expected but missing in final submission (first 10): {list(missing_in_submission)[:10]}")
        # Example check for a specific problematic ID if you know one
        # if any('ID_ZgrspC' in mid for mid in missing_in_submission):
        #      print("One of the problematic 'ID_ZgrspC' IDs is missing.")

submission_filename = f'submission_lfffffdddff_glm0.1_psodo222xgb0.4_ensemble_psl_final.csv' # Updated filename
submission_df_final.to_csv(submission_filename, index=False)
print(f"\nSubmission file '{submission_filename}' created successfully.")
print(f"Number of rows in submission: {len(submission_df_final)}")
print(submission_df_final.head())
print("\nDone!")


--- Ensembling Predictions and Generating Submission (LGBM + GLM + XGB) ---
Debug section 8 - PIDs dtypes before merge 1:
  test_pred_ppm_long['PID']: object
  original_test_bd_df['PID']: object
Debug section 8 - PIDs/Nutrient dtypes before merge 2:
  final_submission_prep_df['PID']: object
  final_submission_prep_df['Nutrient']: object
  gap_test_df_orig['PID']: object
  gap_test_df_orig['Nutrient']: object

Submission file 'submission_lfffffdddff_glm0.1_psodo222xgb0.4_ensemble_psl_final.csv' created successfully.
Number of rows in submission: 26598
            ID          Gap
0  ID_NGS9Bx_N -3418.173747
1  ID_YdVKXw_N -3544.720503
2  ID_MZAlfE_N -3553.700064
3  ID_GwCCMN_N -3426.018703
4  ID_K8sowf_N -4059.313219

Done!
